In [59]:
# TODO make
import os
import io
import re
import matplotlib.pyplot as plt
import pandas as pd

In [33]:
path = r"C:\Users\flahertylab\OneDrive - Georgia Institute of Technology\Flaherty Research Group - ESS\02 - PFR Data\ESS-1-151_Eyring_Plots\05kPa O2\0ppmCl_5kPaO2"
dirlist = os.listdir(path)
data_folders = []
for item in dirlist:
    full_path = os.path.join(path,item)
    if os.path.isdir(full_path) and item.endswith('.D'):
        data_folders.append(item)


In [80]:
def extract_tables_from_file(filepath, start_marker_FID, start_marker_TCD, end_marker):
    with open(filepath, 'r', encoding='utf-16', errors='ignore') as f:
        lines = f.readlines()

    FID_table_lines = []
    TCD_table_lines = []
    inside_table = False
    FID = False
    TCD = False

    for line in lines:
        if start_marker_FID in line:
            inside_table = True
            FID = True
            continue  # skip the start line itself
        elif start_marker_TCD in line:
            inside_table = True
            TCD = True
            continue
        if inside_table:
            if line.strip().startswith(end_marker) and FID and not TCD:  # In FID table
                FID = False
                inside_table = False  # end of table
            elif line.strip().startswith(end_marker) and not FID and TCD: # In TCD table
                TCD = False
                inside_table = False  # end of table
                if FID:
                    print('FID is still inside_table')
                break
            elif FID and not TCD:
                FID_table_lines.append(line.strip())
            else:
                TCD_table_lines.append(line.strip())

    return (FID_table_lines, TCD_table_lines)


In [86]:
file_path = r"C:\Users\flahertylab\OneDrive - Georgia Institute of Technology\Flaherty Research Group - ESS\02 - PFR Data\ESS-1-151_Eyring_Plots\05kPa O2\0ppmCl_5kPaO2\001F0101.D\REPORT.txt"

tables = extract_tables_from_file(
    file_path,
    start_marker_FID="Signal 1: FID1 A,",
    start_marker_TCD="Signal 2: TCD2 B,",
    end_marker="Totals :"
)

# Preview
print("\n".join(tables[1]))
#print("\n".join(TCD_table))



Peak RetTime Type  Width     Area      Height     Area
#   [min]        [min]  [25 uV*s]   [25 uV]       %
----|-------|----|-------|----------|----------|--------|
1   0.397 BB S  0.0356 9149.63770 3799.34985 69.00905
2   0.543 BB X  0.0357 1919.12671  793.68048 14.47458
3   1.499 BB    0.0691  311.79675   70.46693  2.35166
4   2.502 BB    0.1160  889.69891  120.65175  6.71035
5   3.285 BB    0.1584  590.78546   58.22143  4.45586
6   5.027 BB    0.1917  332.99402   24.79430  2.51153
7   8.240 BB    0.1128   64.56530    8.83389  0.48697


In [87]:
def convert_to_dataframe(data_tables):
    # Skip header, units, and separator
    FID_data = data_tables[0]
    TCD_data = data_tables[1]
    data_lines = table_lines[4:]

    parsed_data = []
    for line in data_lines:
        # Regex: split into 7 fields
        match = re.match(
            r"(\d+)\s+([\d.]+)\s+([A-Z]+\s?[A-Z]?)\s+([\d.]+)\s+([\d.eE+-]+)\s+([\d.eE+-]+)\s+([\d.]+)", 
            line
        )
        if match:
            parsed_data.append(match.groups())
        else:
            print(f"Line skipped (couldn't parse): {line}")

    column_names = ['Peak', 'RetTime [min]', 'Type', 'Width', 'Area', 'Height', 'Percent']
    df = pd.DataFrame(parsed_data, columns=column_names)

    # Convert appropriate columns to numeric
    for col in ['Peak', 'RetTime [min]', 'Width', 'Area', 'Height', 'Percent']:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

In [79]:
df = convert_to_dataframe(test)
df.head()

,Peak,RetTime [min],Type,Width,Area,Height,Percent
0,1,0.501,BB,0.0710,2.28594,0.455914,0.01608
1,2,1.664,BB S,0.0266,8596.02734,5261.256350,60.45309
2,3,1.907,BB T,0.0288,5275.73340,2884.626460,37.10253
3,4,4.357,BB,0.0652,2.31922,0.566720,0.01631
4,5,4.863,BB,0.0593,1.36984,0.360590,0.00963
